In [1]:
from architecture import pipe
from PIL import Image
from dataloader import testing_dataset
from util import util
from torch.utils.data import DataLoader
import torch
from torch.nn import BCELoss

D:\ml\mtest\learn_env\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\ml\mtest\learn_env\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\ml\mtest\learn_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
state = torch.load('./saves/exp_3/model_epoch_2.pth')

In [3]:
pipe.model.load_state_dict(state['model_state_dict'])
pipe.model = pipe.model.eval()

In [4]:
dataset = DataLoader(testing_dataset, shuffle=True)

In [31]:
sample = next(iter(dataset))

In [32]:
# util.to_pil(sample[0][0])

In [33]:
print(sample[1])
util.to_classes(sample[1])

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')


(['romance', 'fantasy'], [1.0, 1.0])

In [34]:
res = pipe.pipe(sample[0])
res

tensor([[0.6548, 0.4761, 0.5176, 0.3052, 0.6528, 0.6118, 0.4143, 0.2810, 0.4436,
         0.5586, 0.3530, 0.6499, 0.2798, 0.5317, 0.4556, 0.3418, 0.4390, 0.5181,
         0.5181, 0.4255, 0.4187, 0.2445, 0.5381, 0.4944, 0.3164, 0.4609]],
       device='cuda:0', dtype=torch.float16, grad_fn=<SigmoidBackward0>)

In [35]:
util.pick_top(res[0], 6, prettify=True)

(['action', 'drama', 'martial arts', 'ecchi', 'horror', 'slice of life'],
 [0.65478515625,
  0.65283203125,
  0.64990234375,
  0.61181640625,
  0.55859375,
  0.5380859375])

In [40]:
util.calculate_precession(res, sample[1], cutoff=.6)

tensor([-2.])